# **Récolte des données** 

### TODO : 

 - Finir la description pour les section V et VI
 - Faire une présentation générale au début du NB sur comment celui-ci marche.
 - Changer tous les liens de la section I car tous erronés sauf pour Aix Marseille
 - Retravailler les cellules markdown et les commentaires Python
 - Restructurer les paramètres
 - Pouvoir demander une liste de catégories

### Importation des librairies utiles

In [1]:
import requests
import json
import pandas as pd
import overpy
import geopy.distance

### I. Création du dictionnaire des Localisation avec leur URL respectives

Dans la cellule qui suit, nous récupérons le nom et les statistiques de toutes les villes recensées sur l'application Vigilio.

Puis nous stockons ces informations (uniquement les villes) dans une structure nommée DataFrame de python. Qui est une structure de données bidimensionnelle avec des colonnes de types différents.

In [38]:
#Localisation des données
dict_association_localisation = {"AGGLOMÉRATION LE COTENTIN":'https://vigilo.velorution-cherbourg.fr/get_issues.php?',
                                 "AGGLOMÉRATION TROYENNE":'https://vigilo.troyesenselle.fr/get_issues.php?', 
                                 "AIX MARSEILLE PROVENCE METROPOLE":'https://vigilo.velosenville.org/get_issues.php?',
                                 "BASSIN D'ARCACHON":'https://vigilo.velobassin.org/get_issues.php?',
                                 "BEAUVAISIS":'https://vigilo.vellovaque.fr/get_issues.php?',
                                 "BORDEAUX MÉTROPOLE":'https://vigilo.cycles-manivelles.org/get_issues.php?',
                                 "BREST MÉTROPOLE":'https://vigilo.bapav.org/get_issues.php?',
                                 "EVREUX":'https://vigilo.cube27.org/get_issues.php?',
                                 "GRAND MONTPELLIER":'https://vigilo.jesuisundesdeux.org/get_issues.php?',
                                 "LA ROCHELLE":'https://vigilo.-lr.corolo.fr/get_issues.php?',
                                 "LORIENT AGGLO":'https://vigilo.claav.fr/get_issues.php?',
                                 "MONS":'https://vigilo.geckonline.be/get_issues.php?',
                                 "NANCY AGGLOMÉRATION":'https://vigilo.rigelk.eu/get_issues.php?',
                                 "NANTES MÉTROPOLE":'https://vigilo.placeauvelo-nantes.fr/get_issues.php?',
                                 "NORD-ISÈRE (APIE)":'https://vigilo.apie-asso.net/get_issues.php?',
                                 "PARIS SACLAY":'https://api.vigilo-parissaclay.mdb-idf.org/get_issues.php?',
                                 "PAYS DE CHÂTEAUGIRON":'https://pouliquen.net/vigilo/get_issues.php?',
                                 "PÉRIGUEUX AGGLO":'https://vigilo.velorution-perigourdine.fr/get_issues.php?',
                                 "SAINT MARCELLIN VERCORS ISÈRE COMMUNAUTÉ":'https://vigilo.roulavelo.org/get_issues.php?',
                                 "SAINT-BRIEUC ARMOR AGGLOMÉRATION":'https://api-vigilo.velo-utile.fr/get_issues.php?',
                                 "STRASBOURG":'https://www.strasbourgavelo.eu/get_issues.php?',
                                 "SUD OISE":'https://vigilo.au5v.fr//get_issues.php?',
                                 "THAU":'https://prod-sete.vigilo.velocite-montpellier.fr/get_issues.php?',
                                 "VANNES":'https://vigilo.kaz.bzh/get_issues.php?'
                                }

df_localisation = pd.DataFrame(dict_association_localisation.keys(), columns=["Localisation"])
df_localisation.set_index("Localisation", inplace=True)
df_localisation

""
Localisation
AGGLOMÉRATION LE COTENTIN
AGGLOMÉRATION TROYENNE
AIX MARSEILLE PROVENCE METROPOLE
BASSIN D'ARCACHON
BEAUVAISIS
BORDEAUX MÉTROPOLE
BREST MÉTROPOLE
EVREUX
GRAND MONTPELLIER


### II. Création du dictionnaire avec les différentes catégories de signalements et leur numéro

Cette cellule utilise le module requests pour effectuer une requête HTTP vers une API et récupérer une liste de catégories. 

Ensuite, elle convertit cette liste en un dictionnaire et crée un DataFrame à partir de ce dictionnaire. Le DataFrame affiche les identifiants et les noms des catégories.

In [3]:
# URL de l'API pour récupérer la liste des catégories
url_categories = 'https://vigilo-bf7f2.firebaseio.com/categorieslist.json'
resp = requests.get(url_categories)
resp.raise_for_status()

# Convertir la liste de catégories en un dictionnaire avec le nom de la catégorie comme clé et l'identifiant de catégorie comme valeur
categories_dict = {str(cat["catid"]): cat["catname"] for cat in resp.json()}

# Créer un dataframe à partir du dictionnaire
df_categories = pd.DataFrame(categories_dict.items(), columns=["id", "Catégories"]).set_index("id")

# Afficher le dataframe
df_categories

,Catégories
id,
8,Absence d'aménagement
3,Aménagement mal conçu
4,Défaut d'entretien
5,Absence d'arceaux de stationnement
2,Véhicule ou objet gênant
9,"Accident, chute, incident"
6,"Signalisation, marquage"
7,Incivilité récurrente sur la route
10,Vol ou dégradation de vélo


### III. Liste des POI

Dans la cellule qui suit, nous récupérons l'ensemble des points d'intérêts qui sont disponibles sur Vigilo
pour une ville donnée.

Puis nous stockons ces informations dans un DataFrame de python.

In [7]:
# Créer un dictionnaire des points d'intérêt et leur identifiant OSM (ex: pour les parkings à vélo : "amenity:bicycle_parking")

poi_dict = {
    "Écoles": "amenity=school",
    "Hôpitaux": "amenity=hospital",
    "Parkings à vélos": "amenity=bicycle_parking",
    "Stations de vélos en libre-service": "amenity=bicycle_rental",
    "Passages piétons": "highway=crossing",
    "Feux piétons": "highway=traffic_signals",
    "Places": "highway=pedestrian",
    "Rampes d'accès pour handicapés": "amenity=wheelchair_ramp",
    "Sièges pour handicapés": "amenity=wheelchair_seating",
    "Parkings pour handicapés": "amenity=parking_space",
    "Parcs": "leisure=park",
    "Jardins": "leisure=garden",
    "Terrains de jeux": "leisure=playground",
    "Toilettes publiques": "amenity=toilets",
    "Fontaines à eau": "amenity=drinking_water",
    "Marchés": "amenity=marketplace",
    "Supermarchés": "shop=supermarket",
    "Boulangeries": "shop=bakery",
    "Pharmacies": "amenity=pharmacy",
    "Bibliothèques": "amenity=library",
    "Bureaux de poste": "amenity=post_office",
    "Stations de recharge pour véhicules électriques": "amenity=charging_station",
    "Cliniques": "amenity=clinic",
    "Dentistes": "amenity=dentist",
    "Vétérinaires": "amenity=veterinary",
    "Points de vue": "tourism=viewpoint",
    "Hôtels": "tourism=hotel",
    "Aires de pique-nique": "leisure=picnic_table",
    "Écoles maternelles": "amenity=kindergarten",
    "Écoles primaires": "amenity=primary_school",
    "Écoles secondaires": "amenity=secondary_school"
}

# Créer un dataframe à partir du dictionnaire
df_poi = pd.DataFrame(poi_dict.items(), columns=["Points d'intérêt", "Identifiant OSM"])

# Changer l'index
df_poi = df_poi.set_index("Identifiant OSM")

# Afficher le dataframe
df_poi

,Points d'intérêt
Identifiant OSM,
amenity=school,Écoles
amenity=hospital,Hôpitaux
amenity=bicycle_parking,Parkings à vélos
amenity=bicycle_rental,Stations de vélos en libre-service
highway=crossing,Passages piétons
highway=traffic_signals,Feux piétons
highway=pedestrian,Places
amenity=wheelchair_ramp,Rampes d'accès pour handicapés
amenity=wheelchair_seating,Sièges pour handicapés


### IV. Initialisation des paramètres

La présente section vous permet de choisir les différents paramètres que vous pouvez customiser afin d'effectuer votre requête.

- Définissez la boîte de recherche, pour cela, choississez sur 'site' un point au centre de l'endroit qui vous intéresse (centre de la ville) et entrez la latitude et la longitude de ce point dans central_lat et central_lon respectivement

1. Choisissez une localisation parmi celles proposées dans la liste

    >Pour accéder à la liste des localisations possibles il faut se référer à l'output de la section II.
    
    >Pour que la saisie soit valide merci d'indiquer la localisation entre guillemets : ex `"nom_localisation"`.

In [25]:
# Filtre de recherche par localisation, exemple : localisation = "BREST MÉTROPOLE"
# location_filter = "GRAND MONTPELLIER"
location_filter = "AIX MARSEILLE PROVENCE METROPOLE"

2. Choisissez une date de début ainsi qu'une date de fin

    >Vérifier à ce que la date de début soit bien inférieur à la date de fin
    
    >Pour que la saisie soit valide merci d'indiquer la date avec le format suivant : `"YYYY-MM-DD"`.

In [21]:
# FIltre de recherche par date, exemple : date_filter = "2021-01-01" dans le format "YYYY-MM-DD hh:mm:ss"
date_debut = "2000-01-01 00:00:00"
date_fin = "2024-12-31 23:59:59"

3. Choisissez une catégorie parmi celles proposées dans la liste

    >Pour accéder à la liste des catégories possibles il faut se référer à l'output de la section III.
    
    >Pour que la saisie soit valide merci d'indiquer l'identifiant de la catégorie (**et non le nom de celle-ci**) entre guillemets : ex `"8"` (pour indiquer la catégorie Absence d'aménagement).

In [31]:
# Filtre des catégories à afficher, exemple : category_filter = "8" donne la catégorie Absence d'aménagement
# category_filter = None
category_filter = "8"

4. Choisissez un ou plusieurs points d'intérêt (POI) parmi ceux proposés dans la liste

    >Pour accéder à la liste des catégories possibles il faut se référer à l'output de la section VI.
    
    >Pour que la saisie soit valide merci d'indiquer les POIs entre guillemets, puis le tout entre crochets : ex `["Écoles", "Parkings à vélos"]`

In [23]:
# # Paramètres de recherche

# # Filtre de recherche par localisation, exemple : localisation = "BREST MÉTROPOLE"
# location_filter = "GRAND MONTPELLIER"

# # FIltre de recherche par date, exemple : date_filter = "2021-01-01" dans le format "YYYY-MM-DD hh:mm:ss"
# date_debut = "2000-01-01 00:00:00"
# date_fin = "2024-12-31 23:59:59"

# # Filtre des catégories à afficher, exemple : category_filter = "8" donne la catégorie Absence d'aménagement
# category_filter = None

# # Filtre des points d'intérêt à afficher, exemple : poi_filter = "Parking à vélo" donne le point d'intérêt Parking à vélo
poi_names = ["Écoles", "Écoles maternelles", "Écoles primaires", "Écoles secondaires"]

#Boîte de recherche pour les points d'intérêt exemple Brest : (48.390177 -4.496499)
central_lat = 43.616449 
central_lon = 3.869016 
delta = 0.02 

### V. Création du dataframe des données

Dans la cellule qui suit, nous récupérons l'ensemble des points d'intérêts qui sont disponibles sur Vigilo
grâce à une requête vers leur API.

La requête est composée des paramètres initialisés dans la cellule précédente.

Un formatage spécifique est réservé pour les données en temps que l'on récupère afin qu'elles soient 
lisibles et exploitables par la suite.

Puis certaines des informations sont renommées et d'autres sont supprimées afin de n'avoir que les informations
ayant une plus value pour l'utilisateur.

In [39]:
def data_collection(location_filter, category_filter, date_debut=None, date_fin=None):
    try:
        # Construct the URL for the API call
        url = dict_association_localisation[location_filter] + ('&c=' + category_filter if category_filter is not None else '')
        resp = requests.get(url)
        resp.raise_for_status()  # Raises an exception for HTTP errors

        # Load data into DataFrame
        df = pd.DataFrame(resp.json())

        # Ensure 'time' column is treated as integer for Unix timestamp conversion
        # Explicitly cast to integer before conversion
        df['time'] = pd.to_datetime(df['time'].astype(int), unit='s')

        # Convert datetime back to Unix timestamp for comparison (if filtering is needed)
        if date_debut is not None and date_fin is not None:
            date_debut_dt = pd.to_datetime(date_debut).timestamp()
            date_fin_dt = pd.to_datetime(date_fin).timestamp()
            df = df[(df['time'].astype('int64') // 10**9 >= date_debut_dt) & 
                    (df['time'].astype('int64') // 10**9 <= date_fin_dt)]

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except ValueError as e:
        print(f"Error parsing JSON or date conversion issue: {e}")
        return None
df = data_collection(location_filter, category_filter, date_debut, date_fin)
df.rename(columns={"coordinates_lat": "latitude", "coordinates_lon": "longitude"}, inplace=True)
df.drop(columns=["status", "group", "approved"], inplace=True)

### Aperçu des données récoltées

In [42]:
df.head()

,token,latitude,longitude,address,comment,explanation,time,categorie,cityname
0,3FB496C0,43.24272843204604,5.4026659950613976,"Boulevard Poméon, Marseille",Pas de visibilité sur le voitures sortant du boul,Pas de visibilité sur les véhicules sortant du...,2023-05-09 06:34:00,8,Marseille
1,CA6305B6,43.25663697862776,5.382781773805619,"Avenue Clot-Bey, Marseille",pas de solution de parking vélo,,2023-05-04 08:46:00,8,Marseille
2,8BF9D2C2,43.23166922069623,5.436846725642681,"Avenue de Luminy, Marseille",aucun aménagement sur université,,2023-04-22 13:21:00,8,Marseille
3,876A8C89,43.24949164401673,5.404922738671302,"Avenue de Lattre de Tassigny, Marseille",secteur dangereux sans aménagement,,2023-04-22 12:49:00,8,Marseille
4,A2822899,43.30118452057706,5.391417145729065,Boulevard de la Libération - Général de Monsab...,Absence d'un M12fd qui serait justifié,un panneau M12 faisant dérogation aux velos po...,2023-02-20 16:30:00,8,Marseille


### VI. Récupération des données OpenStreetMap autour de la région cherchée

TODO : refaire description

Cette cellule contient du code qui utilise les packages pandas, requests et json pour récupérer des points d'intérêt (POI) à partir d'une API Overpass. Les POI sont filtrés en fonction d'une zone géographique définie par les coordonnées bbox. Les données récupérées sont ensuite traitées et stockées dans un dataframe pandas.

In [43]:
bbox = [
    central_lat - delta,  # south latitude
    central_lon - delta,  # west longitude
    central_lat + delta,  # north latitude
    central_lon + delta   # east longitude
]

def fetch_poi_data(bbox, poi_names, poi_dict):
    # Prepare the Overpass query part for multiple POI tags
    query_parts = []
    for poi_name in poi_names:
        poi_tag = poi_dict.get(poi_name)
        if poi_tag is None:
            raise ValueError(f"Le point d'intérêt '{poi_name}' n'est pas reconnu.")
        # Format tag correctly for the Overpass query
        formatted_poi_tag = poi_tag.replace("=", '="') + '"'
        query_part = f"""
            node[{formatted_poi_tag}](bbox:{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
            way[{formatted_poi_tag}](bbox:{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
            rel[{formatted_poi_tag}](bbox:{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
        """
        query_parts.append(query_part)
    
    # Combine all query parts
    full_query = f"""
    [out:json];
    (
        {''.join(query_parts)}
    );
    out center;
    """
    
    # Define the Overpass API URL
    url = "http://overpass-api.de/api/interpreter"

    # Send the request to the Overpass API
    response = requests.get(url, params={'data': full_query})
    response.raise_for_status()  # Raises an HTTPError for bad responses
    return response.json()

def process_poi_data(json_data):
    # Initialize a dictionary to hold all POIs
    poi_dict = {}
    
    # Go through each element in the JSON data
    for element in json_data['elements']:
        # Check if the element is a node and has the required 'tags'
        if element['type'] == 'node' and 'tags' in element:
            # Check if the latitude and longitude are within the bounding box
            if bbox[0] <= element['lat'] <= bbox[2] and bbox[1] <= element['lon'] <= bbox[3]:
                poi_info = {
                    'name': element['tags'].get('name', 'Unknown'),
                    'latitude': element.get('lat', None),
                    'longitude': element.get('lon', None),
                    }
                
                # Add this POI's information to the main dictionary
                poi_dict[element['id']] = poi_info
            
    return poi_dict

pois_json = fetch_poi_data(bbox, poi_names, poi_dict)
pois = process_poi_data(pois_json)
df_poi = pd.DataFrame.from_dict(pois, orient='index')

### Combinaison des données Vigilo et OpenStreetMap

TODO : refaire description

Cette cellule de code définit une fonction en Python appelée calculate_distance qui prend en entrée les coordonnées de deux points géographiques (latitude et longitude) et renvoie la distance entre ces deux points en mètres. La fonction utilise la bibliothèque geopy pour effectuer le calcul de distance.

In [44]:
# Saving the data to a CSV file in the data_analysis folder
df.to_csv("df_incidents.csv", index=False)
df_poi.to_csv("df_poi.csv", index=True)